### Importowanie pakietów z znanym nam zbiorem iris

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [5]:
type(iris)

sklearn.utils.Bunch

In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [9]:
# !pip install sqlalchemy

### Tworzenie i łączenie się z bazą danych SQL

Polecenie create_engine pozwala nam się podłączyć do już istniejącej bazy danych.
SQLite to baza danych "plikowa". Stworzy nam się plik .db, który będzie zawierałnasze bazy danych

In [11]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///irysy4.db')
baza_danych = create_engine('sqlite:///irysy5.db')

Za pomocą polecenia to_sql możemy zapisać nasze obiekty jako tabele sql-owe i zapisać w bazie danych

In [12]:
df.to_sql('dane', con=engine, index=False)

In [13]:
a = engine.execute("SELECT * FROM dane").fetchall()

In [15]:
engine.execute("SELECT AVG([sepal width (cm)]) FROM dane").fetchall()

[(3.057333333333334,)]

In [16]:
type(a)

list

In [18]:
a[0]

(5.1, 3.5, 1.4, 0.2, 0.0)

In [19]:
type(a[0])

sqlalchemy.engine.row.LegacyRow

Te dane możemy skonwertować do data frame

In [20]:
pd.DataFrame(a)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [21]:
df2 = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df2

,name
0,User 1
1,User 2
2,User 3


In [23]:
df2.to_sql('users', con=engine, if_exists="replace",index=True)
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3')]

In [24]:
df3 = pd.DataFrame({'name' : ['User 6', 'User 7']})
df3.to_sql('users', con=engine, if_exists='append')

In [25]:
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3'), (0, 'User 6'), (1, 'User 7')]

In [26]:
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [27]:
engine = create_engine('sqlite:///irysy2.db')
base = declarative_base()

In [28]:
type(base)

sqlalchemy.orm.decl_api.DeclarativeMeta

In [29]:
class Transakcje(base):
    
    __tablename__ = 'transakcje'
    
    transakcja_id = Column(Integer, primary_key=True)
    data = Column(String)
    przedmiot_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, transakcja_id, data, przedmiot_id, cena):
        self.transakcja_id = transakcja_id
        self.data = data
        self.przedmiot_id = przedmiot_id
        self.cena = cena

In [30]:
class Kwiaty(base):
    
    __tablename__ = 'irysy'
    
    kwiat_id   = Column(Integer, primary_key=True)
    dzial_dlug = Column(Integer)
    dzial_szer = Column(Integer)
    plat_dlug  = Column(Integer)
    plat_szer  = Column(Integer)
    
    def __init__(self, kwiat_id, dzial_dlug, dzial_szer, plat_dlug, plat_szer):
        self.kwiat_id = kwiat_id
        self.dzial_dlug = dzial_dlug
        self.dzial_szer = dzial_szer
        self.plat_dlug = plat_dlug
        self.plat_szer = plat_szer

In [31]:
# tworzenie tabel
base.metadata.create_all(engine)

In [32]:
from sqlalchemy.orm import sessionmaker
# Stworzenie nowej sesji
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(10):
    tr = Transakcje(t, f'200{t}/05/06', t**2-t*2, 19)
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [33]:
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(df.shape[0]):
    tr = Kwiaty(t, df.iloc[t,0], df.iloc[t,1], df.iloc[t,2],df.iloc[t,3])
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [37]:
# engine.execute("SELECT * FROM irysy").fetchall()

In [35]:
engine.execute("SELECT * FROM Transakcje").fetchall()

[(0, '2000/05/06', 0, 19),
 (1, '2001/05/06', -1, 19),
 (2, '2002/05/06', 0, 19),
 (3, '2003/05/06', 3, 19),
 (4, '2004/05/06', 8, 19),
 (5, '2005/05/06', 15, 19),
 (6, '2006/05/06', 24, 19),
 (7, '2007/05/06', 35, 19),
 (8, '2008/05/06', 48, 19),
 (9, '2009/05/06', 63, 19)]

In [36]:
engine.execute("SELECT * FROM transakcje").fetchall()

[(0, '2000/05/06', 0, 19),
 (1, '2001/05/06', -1, 19),
 (2, '2002/05/06', 0, 19),
 (3, '2003/05/06', 3, 19),
 (4, '2004/05/06', 8, 19),
 (5, '2005/05/06', 15, 19),
 (6, '2006/05/06', 24, 19),
 (7, '2007/05/06', 35, 19),
 (8, '2008/05/06', 48, 19),
 (9, '2009/05/06', 63, 19)]

In [38]:
# wszystkie dane
for s in session.query(Transakcje).all():
    print(s.transakcja_id, s.data)

0 2000/05/06
1 2001/05/06
2 2002/05/06
3 2003/05/06
4 2004/05/06
5 2005/05/06
6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [39]:
# wybrane transakcje
for s in session.query(Transakcje).filter(Transakcje.transakcja_id>5):
    print(s.transakcja_id, s.data)

6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [40]:
# wybrane Kwiaty
for s in session.query(Kwiaty).filter(Kwiaty.dzial_dlug>5):
    print(s.kwiat_id, s.dzial_dlug)

0 5.1
5 5.4
10 5.4
14 5.8
15 5.7
16 5.4
17 5.1
18 5.7
19 5.1
20 5.4
21 5.1
23 5.1
27 5.2
28 5.2
31 5.4
32 5.2
33 5.5
36 5.5
39 5.1
44 5.1
46 5.1
48 5.3
50 7
51 6.4
52 6.9
53 5.5
54 6.5
55 5.7
56 6.3
58 6.6
59 5.2
61 5.9
62 6
63 6.1
64 5.6
65 6.7
66 5.6
67 5.8
68 6.2
69 5.6
70 5.9
71 6.1
72 6.3
73 6.1
74 6.4
75 6.6
76 6.8
77 6.7
78 6
79 5.7
80 5.5
81 5.5
82 5.8
83 6
84 5.4
85 6
86 6.7
87 6.3
88 5.6
89 5.5
90 5.5
91 6.1
92 5.8
94 5.6
95 5.7
96 5.7
97 6.2
98 5.1
99 5.7
100 6.3
101 5.8
102 7.1
103 6.3
104 6.5
105 7.6
107 7.3
108 6.7
109 7.2
110 6.5
111 6.4
112 6.8
113 5.7
114 5.8
115 6.4
116 6.5
117 7.7
118 7.7
119 6
120 6.9
121 5.6
122 7.7
123 6.3
124 6.7
125 7.2
126 6.2
127 6.1
128 6.4
129 7.2
130 7.4
131 7.9
132 6.4
133 6.3
134 6.1
135 7.7
136 6.3
137 6.4
138 6
139 6.9
140 6.7
141 6.9
142 5.8
143 6.8
144 6.7
145 6.7
146 6.3
147 6.5
148 6.2
149 5.9


In [46]:
import json

JSON - Java Script Object Notation (ale w żaden sposób nie jest połączony z Java Scriptem czy żadnym innym językiem).  
JSON to sposób na lekki format wymiany danych.  
W JSON są czyste dane, nie ma tam jakichś zmiennych czy fukncji. Popularyazcja jego nastąpiła od mniej więcej 2001. Teraz jest używanie prawie, że domyślnie do wymiany danych między aplikacjami.

In [47]:
from IPython.display import HTML
from IPython.display import Image

In [48]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/haYYypSnOTY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [49]:
person = '{"name": "Bob", "languages": ["English", "French"]}'
type(person)

str

In [50]:
person

'{"name": "Bob", "languages": ["English", "French"]}'

In [51]:
person_dict = json.loads(person)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(person_dict)

# Output: ['English', 'French']
print(person_dict['languages'])

{'name': 'Bob', 'languages': ['English', 'French']}
['English', 'French']


In [52]:
person_dict

{'name': 'Bob', 'languages': ['English', 'French']}

In [53]:
type(person_dict)

dict

In [54]:
%%file test5.json


{"name": "Bob", 
"languages": ["English", "French"]
}

Writing test5.json


In [55]:
with open('test5.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data)

{'name': 'Bob', 'languages': ['English', 'French']}


In [56]:
type(data)

dict

In [57]:
data['name']

'Bob'

In [58]:
data['languages']

['English', 'French']

In [59]:
person_dict = {'name': 'Bob',
'age': 12,
'children': None
}

person_json = json.dumps(person_dict)
print(person_json)

{"name": "Bob", "age": 12, "children": null}


In [60]:
with open('person2.json', 'w') as json_file:
    json.dump(person_dict, json_file)

In [61]:
print(json.dumps(person_dict, indent = 4, sort_keys=True))

{
    "age": 12,
    "children": null,
    "name": "Bob"
}


In [63]:
# !pip install flask
# !pip install flask_restful
# !pip install flask_jsonpify

In [64]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify

In [65]:
app  = Flask(__name__)
api = Api(app)

@app.route('/')
@app.route('/index')
def home():
    #return render_template('home.html')
    return "Strona poczatkowa"

@app.route('/hello/<name>')
def success(name):
    return f'<h1>{name}</h1>'

In [66]:
class Main(Resource):
    def get(self):
        return jsonify("Hello world")

In [67]:
api.add_resource(Main,'/test')

In [68]:
class Irys(Resource):
    
    def get(self):
        conn=engine.connect()
        query = conn.execute('select * from irysy')
        result = {'dane':[i for i in query.cursor.fetchall()]}
        return jsonify(result)

In [69]:
api.add_resource(Irys, '/irys')

In [70]:
app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2021 15:23:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 15:23:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2021 15:24:32] "GET /f HTTP/1.1" 404 -
